# Hint Strategy 1
This notebook demonstrates how an LLM performs when presented with a Javaee -> Quarkus JMS issue and provide a hint that can be used to improve the basic rule.
Installing pre requisites and configuring Kai with `GPT-4o-mini`

In [1]:
%pip uninstall kai -y
%pip install --no-cache-dir git+https://github.com/konveyor/kai.git@main
%pip install python-dotenv

Found existing installation: kai 0.1.1rc0
Uninstalling kai-0.1.1rc0:
  Successfully uninstalled kai-0.1.1rc0
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/konveyor/kai.git (to revision main) to /private/var/folders/4f/bv2hh9jd44b94_4zf2qby2sh0000gn/T/pip-req-build-gfrckgch
  Running command git clone --filter=blob:none --quiet https://github.com/konveyor/kai.git /private/var/folders/4f/bv2hh9jd44b94_4zf2qby2sh0000gn/T/pip-req-build-gfrckgch
  Resolved https://github.com/konveyor/kai.git to commit 923abc35262ed0e36094c91ee20f2e0cabc3bbf5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kai: filename=kai-0.1.1rc0-py3-none-any.whl size=4435614 sha256=76ce116eb0d7ef8cd97d6b355b055f127c8225270709a5899b98206393bf1d36
  Stored in directory: /private/var/folders/4f/bv2hh9jd44b94_4zf2qby2sh0000gn/T/pip-ephem-wheel-cache-wz90ghbx/wheels/7a/5a/

In [2]:
from IPython.display import display, Markdown
from kai.llm_interfacing.model_provider import ModelProvider
from kai.kai_config import KaiConfigModels, SupportedModelProviders
from dotenv import load_dotenv
import os
load_dotenv(override=True) 

# Initialize the model provider 
model = ModelProvider.from_config(KaiConfigModels(
    provider=SupportedModelProviders.CHAT_OPENAI,
    args={"model": "granite-8b-code-instruct-128k",
        "base_url": "https://granite-8b-code-instruct-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443/v1",
        "api_key": os.getenv("OPENAI_API_KEY"),
        "temperature": 0.1, },
))

# Async rendering function for displaying the response
async def rendered_llm_call(prompt: str):
    response = await model.ainvoke_llm(prompt)
    display(Markdown(response.content))
    return response


/Users/sraghuna/local_dev/kai-notebook-solution-server/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Below is the snippet we are trying to migrate from javaee to Qusrkus. This code uses JMS which needs to be migarted to Quarkus equivalent.
Our goal is to assess how well the LLM refactors this configuration without a hint, and whether it can replace deprecated patterns with the right approach.

In [3]:
before_code = """\
import javax.jms.* 

public class InventoryNotificationMDB implements MessageListener {

    private static final int LOW_THRESHOLD = 50;

    @Inject
    private CatalogService catalogService;

    private final static String JNDI_FACTORY = "weblogic.jndi.WLInitialContextFactory";
    private final static String JMS_FACTORY = "TCF";
    private final static String TOPIC = "topic/orders";
    private TopicConnection tcon;
    private TopicSession tsession;
    private TopicSubscriber tsubscriber;

    public void onMessage(Message rcvMessage) {
        TextMessage msg;
        {
            try {
                System.out.println("received message inventory");
                if (rcvMessage instanceof TextMessage) {
                    msg = (TextMessage) rcvMessage;
                    String orderStr = msg.getBody(String.class);
                    Order order = Transformers.jsonToOrder(orderStr);
                    order.getItemList().forEach(orderItem -> {
                        int old_quantity = catalogService.getCatalogItemById(orderItem.getProductId()).getInventory().getQuantity();
                        int new_quantity = old_quantity - orderItem.getQuantity();
                        if (new_quantity < LOW_THRESHOLD) {
                            System.out.println("Inventory for item " + orderItem.getProductId() + " is below threshold (" + LOW_THRESHOLD + "), contact supplier!");
                        } else {
                            orderItem.setQuantity(new_quantity);
                        }
                    });
                }


            } catch (JMSException jmse) {
                System.err.println("An exception occurred: " + jmse.getMessage());
            }
        }
    }
    }
"""

This rule flags suggests migrating javaee JMS referesnces with Quarkus specific ones.

In [4]:
hint_to_migrate="""\
    
  description:  References to JavaEE/JakartaEE JMS elements should be removed and replaced with their Quarkus SmallRye/Microprofile equivalents.
  message: |
   References to JavaEE/JakartaEE JMS elements should be removed and replaced with their Quarkus SmallRye/Microprofile equivalents.

    """

In [5]:
source="javaee"
target="quarkus"

In [6]:
prompt = f"""

The following java code needs to be migrated from {source} to {target}.

```java
{before_code}
```

Use the following hint's description and message to migrate. 

{hint_to_migrate}

Also, remove the deprecated and unused APIs and suggest the correct alternatives based on the migration needs. 


Please provide a refactored version of {before_code} that is compatible with {target}. 
Output: 

* Refactored Code in Java

* Step-by-Step Migration Process

* New Hint Summary to guide future similar migrations. Include optional changes that may be needed

"""

In [7]:

response = await rendered_llm_call(prompt)

Here is a refactored version of the code that is compatible with Quarkus:

```java
import io.smallrye.reactive.messaging.annotations.Channel;
import io.smallrye.reactive.messaging.annotations.Emitter;

public class InventoryNotificationMDB {

    private static final int LOW_THRESHOLD = 50;

    @Inject
    private CatalogService catalogService;

    @Emitter("orders")
    private Emitter<String> orderEmitter;

    public void onMessage(String orderStr) {
        Order order = Transformers.jsonToOrder(orderStr);
        order.getItemList().forEach(orderItem -> {
            int old_quantity = catalogService.getCatalogItemById(orderItem.getProductId()).getInventory().getQuantity();
            int new_quantity = old_quantity - orderItem.getQuantity();
            if (new_quantity < LOW_THRESHOLD) {
                System.out.println("Inventory for item " + orderItem.getProductId() + " is below threshold (" + LOW_THRESHOLD + "), contact supplier!");
            } else {
                orderItem.setQuantity(new_quantity);
                orderEmitter.send(orderStr);
            }
        });
    }
}
```

This code uses the SmallRye Reactive Messaging API to send messages to a topic named "orders". The `onMessage` method takes a `String` parameter representing the JSON-encoded order, and updates the inventory for each item in the order. If the new quantity is below the threshold, it prints a message and does not send the updated order. Otherwise, it sends the updated order to the topic.

To migrate this code to Quarkus, you will need to make the following changes:

1. Replace the `@Inject` annotation with the `@Channel` annotation to inject the `orderEmitter` into the class.

2. Replace the `@Emitter` annotation with the `@Channel` annotation to define the "orders" channel.

3. Remove the `JNDI_FACTORY`, `JMS_FACTORY`, and `TOPIC` constants, as they are not needed in Quarkus.

4. Remove the `TopicConnection`, `TopicSession`, and `TopicSubscriber` fields, as they are not needed in Quarkus.

5. Remove the `onMessage` method, as it is not needed in Quarkus.

6. Replace the `TextMessage` and `Body` method with the `String` and `Body` method, respectively.

7. Replace the `Transformers.jsonToOrder` method with the `Jsonb.fromJson` method to convert the JSON-encoded order to a `Order` object.

8. Replace the `catalogService.getCatalogItemById` method with the `catalogService.getCatalogItemById` method to get the catalog item by ID.

9. Replace the `old_quantity` and `new_quantity` variables with the `oldQuantity` and `newQuantity` variables, respectively.

10. Replace the `System.out.println` method with the `LOGGER.info` method to log messages.

11. Replace the `orderEmitter.send` method with the `orderEmitter.send` method to send the updated order to the topic.

Here is a step-by-step migration process for this code:

1. Add the SmallRye Reactive Messaging extension to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

2. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

3. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

4. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

5. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

6. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

7. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

8. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

9. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

10. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

11. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

12. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

13. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

14. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

15. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

16. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

17. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

18. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

19. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

20. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

21. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

22. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

23. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

24. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

25. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

26. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

27. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

28. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

29. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

30. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

31. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

32. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

33. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

34. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

35. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

36. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

37. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

38. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

39. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

40. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

41. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

42. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

43. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

44. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

45. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

46. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

47. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

48. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

49. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

50. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

51. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

52. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

53. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

54. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```

55. Add the `smallrye-reactive-messaging-kafka` dependency to the project's `pom.xml` file:

```xml
<dependency>
    <groupId>io.smallrye</groupId>
    <artifactId>smallrye-reactive-messaging-kafka</artifactId>
    <version>2.0.0</version>
</dependency>
```